In [3]:
import string
from nltk.corpus import stopwords
import pandas as pd

In [31]:
df = pd.read_csv("C://Users//manka//Documents//Predictive Analytics//Project//Final//final_data.csv")
del df['Unnamed: 0']

In [32]:
def remove(tweet):

    noPunctTweet = [char for char in tweet if char not in string.punctuation]

    noPunctTweet = ''.join(noPunctTweet)
    
    return [word for word in noPunctTweet.split() if word.lower() not in stopwords.words('english')]

In [33]:
df['title'].head(5).apply(remove)

0    [BREAKING, First, NFL, Team, Declares, Bankrup...
1    [Court, Orders, Obama, Pay, 400, Million, Rest...
2    [UPDATE, Second, Roy, Moore, Accuser, Works, M...
3        [Oscar, Pistorius, Attempts, Commit, Suicide]
4                  [Trump, Votes, Death, Penalty, Gay]
Name: title, dtype: object

In [34]:
df.head()

,news_url,title,no_of_retweets,label,length
0,speedtalk.com,BREAKING: First NFL Team Declares Bankruptcy O...,163,1,64
1,politics2020.info,Court Orders Obama To Pay $400 Million In Rest...,102,1,53
2,nscdscamps.org,UPDATE: Second Roy Moore Accuser Works For Mic...,220,1,69
3,howafrica.com,Oscar Pistorius Attempts To Commit Suicide,22,1,42
4,washingtonsources.org,Trump Votes For Death Penalty For Being Gay,550,1,43


In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
bow_transformer1 = CountVectorizer(analyzer=remove).fit(df['title'])
bow_transformer2 = CountVectorizer(analyzer=remove).fit(df['news_url'])

print(len(bow_transformer1.vocabulary_))
print(len(bow_transformer2.vocabulary_))

9479
1356


In [56]:
tweets_bow = bow_transformer1.transform(df['title'])
news_url_bow = bow_transformer2.transform(df['news_url'])

In [57]:
print('Shape of Sparse Matrix: ', tweets_bow.shape)
print('Amount of Non-Zero occurences: ', tweets_bow.nnz)

Shape of Sparse Matrix:  (4057, 9479)
Amount of Non-Zero occurences:  37214


In [58]:
print('Shape of Sparse Matrix: ', news_url_bow.shape)
print('Amount of Non-Zero occurences: ', news_url_bow.nnz)

Shape of Sparse Matrix:  (4057, 1356)
Amount of Non-Zero occurences:  4057


In [39]:
from sklearn.feature_extraction.text import TfidfTransformer

In [59]:
tfidf_transformer1 = TfidfTransformer().fit(tweets_bow)
tfidf_transformer2 = TfidfTransformer().fit(news_url_bow)

In [61]:
tweets_tfidf = tfidf_transformer1.transform(tweets_bow)
news_url_tfidf = tfidf_transformer2.transform(news_url_bow)
print(tweets_tfidf.shape)
print(news_url_tfidf.shape)

(4057, 9479)
(4057, 1356)


In [62]:
df1 = pd.DataFrame(tweets_tfidf.toarray())
df2 = pd.DataFrame(news_url_tfidf.toarray())

In [68]:
df3 = df.drop(['news_url','title', 'label'], axis=1)

In [69]:
df3.head()

,no_of_retweets,length
0,163,64
1,102,53
2,220,69
3,22,42
4,550,43


In [70]:
finaldf = pd.concat([df1,df2,df3], axis=1)
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4057 entries, 0 to 4056
Columns: 10837 entries, 0 to length
dtypes: float64(10835), int64(2)
memory usage: 335.4 MB


In [71]:
df.head()

,news_url,title,no_of_retweets,label,length
0,speedtalk.com,BREAKING: First NFL Team Declares Bankruptcy O...,163,1,64
1,politics2020.info,Court Orders Obama To Pay $400 Million In Rest...,102,1,53
2,nscdscamps.org,UPDATE: Second Roy Moore Accuser Works For Mic...,220,1,69
3,howafrica.com,Oscar Pistorius Attempts To Commit Suicide,22,1,42
4,washingtonsources.org,Trump Votes For Death Penalty For Being Gay,550,1,43


In [74]:
from sklearn.model_selection import train_test_split

In [75]:
X_train, X_test, y_train, y_test = train_test_split(finaldf, df['label'], test_size=0.3, random_state=101)

## Naive Bayes

In [77]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(X_train, y_train)

In [78]:
predictions = nb_model.predict(X_test)

In [79]:
from sklearn.metrics import classification_report, confusion_matrix

In [80]:
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[148 584]
 [117 369]]
              precision    recall  f1-score   support

           0       0.56      0.20      0.30       732
           1       0.39      0.76      0.51       486

    accuracy                           0.42      1218
   macro avg       0.47      0.48      0.40      1218
weighted avg       0.49      0.42      0.38      1218



## Support Vector Machines

In [83]:
from sklearn.svm import SVC

In [84]:
svm_model = SVC()

In [85]:
svm_model.fit(X_train,y_train)

C:\Users\manka\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [86]:
predictions = svm_model.predict(X_test)

In [87]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test, predictions))

[[726   6]
 [477   9]]
              precision    recall  f1-score   support

           0       0.60      0.99      0.75       732
           1       0.60      0.02      0.04       486

    accuracy                           0.60      1218
   macro avg       0.60      0.51      0.39      1218
weighted avg       0.60      0.60      0.47      1218



## K Means Clustering

In [88]:
from sklearn.cluster import KMeans

In [89]:
kmeans_model = KMeans(n_clusters=2)

In [90]:
kmeans_model.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [91]:
predictions = kmeans_model.predict(X_test)

In [92]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test, predictions))

[[732   0]
 [486   0]]
              precision    recall  f1-score   support

           0       0.60      1.00      0.75       732
           1       0.00      0.00      0.00       486

    accuracy                           0.60      1218
   macro avg       0.30      0.50      0.38      1218
weighted avg       0.36      0.60      0.45      1218



C:\Users\manka\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
